In [1]:
import json
import numpy as np
import pandas as pd
import re

In [2]:
data = []
with open("test.json", "r", encoding="utf-8") as f:
    for line in f.readlines():
        data.append(json.loads(line))
        
data = pd.DataFrame(data)
data.head()

,id,last_analyzed,document_id,size,content_type,doc_title,agenda_item_id,agenda_item_title,agenda_item_text,meeting_id,...,creation_date,last_modified,last_saved,doc_type,consultation_id,consultation_name,consultation_type,consultation_text,consultation_topic,consultation_organization
0,1,2023-05-06T22:57:49Z,6164,169369,application/pdf,Stvv01 - 14.04.2011,[],[],[],[1413],...,2017-11-30T12:08:04Z,2017-11-30T12:08:04Z,2017-11-30T12:08:04Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2023-05-06T22:57:49Z,553,99102,application/pdf,1 Pro-JSKS,[],[],[],[69],...,2016-05-26T05:49:28Z,2016-05-26T05:49:28Z,2016-05-26T05:49:28Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2023-05-06T22:57:49Z,722,183796,application/pdf,Kostenplan 2013-12 Mai 2016,[],[],[],[76],...,2016-05-10T07:46:01Z,2016-05-10T09:32:42Z,2016-05-10T09:32:42Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2023-05-06T22:57:49Z,3961,331258,application/pdf,Niederschrift SB,[],[],[],[1220],...,2017-03-20T14:30:09Z,2017-03-20T14:30:09Z,2017-03-20T14:30:09Z,Niederschrift,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2023-05-06T22:57:49Z,723,267877,application/pdf,2 Pro-Schw,[],[],[],[76],...,2016-08-12T09:23:49Z,2016-08-12T09:23:49Z,2016-08-12T09:23:49Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
data.doc_type.unique()

array([nan, 'Niederschrift', 'Beschlussvorlage', 'Antragsvorlage',
       'Informationsvorlage'], dtype=object)

In [112]:
def iterate_content(doc_type=None, document_ids=None):
    if doc_type:
        filtered = data[data.doc_type == doc_type]
    else:
        filtered = data
    
    if document_ids:
        filtered = filtered[np.isin(filtered.document_id, document_ids)]
    
    for (document_id, content) in zip(filtered["document_id"], filtered["content"]):
        yield (document_id, content)
        

def remove_by_regexes(content, regexes, from_end=False):
    """ Removes the header of documents based on given regex patterns.
    The content lines are scanned for the first occurance of all regexes and the returned output
    contains only the content after the last regex match.
    Therefore, if a regex is found twice, only the first match is considered.
    If two regexes match, the content is cut for the last match of the two.
    """
    matches = []  # tuples (line, regex_start, regex_end) for all first matches
    for regex in regexes:
        for idx, line in enumerate(content):
            res = re.search(regex, line, re.IGNORECASE)
            if res:
                matches.append((idx, res.start(), res.end()))
                break
   
    if not from_end:
        # find the last line containing a regex
        last_match_line = np.max([match[0] for match in matches])
        
        # in the last line, find the latest end of a regex in that line
        last_match_inline = 0
        for line, start, end in matches:
            if line != last_match_line:
                continue
            last_match_inline = np.max([last_match_inline, end])
        
        # strip content from everything before the latest occcurance of the match
        content = content[last_match_line:]
        content[0] = content[0][last_match_inline:]
        if content[0].strip() == "":
            content = content[1:]
    else:
        # find the first line containing a regex
        first_match_line = np.min([match[0] for match in matches])
        
        # in the first line, find the first start of a regex in that line
        first_match_inline = len(content[first_match_line])
        for line, start, end in matches:
            if line != first_match_line:
                continue
            first_match_inline = np.min([first_match_inline, start])
            
        # strip content from everything before the latest occcurance of the match
        content = content[:first_match_line+1]
        content[-1] = content[-1][:first_match_inline]
        if content[-1].strip() == "":
            content = content[:-1]

    return content

# test remove_by_regexes:
result = remove_by_regexes([
    "first line",
    "second line with match in middle",
    "third line with match in middle",
    "fourth line"
], [r"with match", "third line"])
assert result == [" with match in middle", "fourth line"]

# test remove from end
result = remove_by_regexes([
    "first line",
    "second line with match in middle",
    "third line with match in middle",
    "fourth line"
], [r"with match", "third line"], True)
assert result == ["first line", "second line "]


# Antragsvorlage

In [38]:
# create a test dataset for antrag header removal test
antraege = data[data.doc_type == "Antragsvorlage"]
print("dataset size", len(antraege))
data_set_identifiers = [
    lambda x: "die Stadtverordnetenversammlung möge beschließen:" in x and x.endswith(":"),
    lambda x: "die Stadtverordnetenversammlung möge beschließen" in x and "beschließen:" not in x,
    lambda x: "die Stadtverordnetenversammlung möge beschließen" in x and len(x) > 100,
    lambda x: "Wir beantragen. die Stadtverordnetenversammlung möge beschließen" in x,
    lambda x: "bitte nehmen Sie den folgenden Antrag zur Beratungen auf die Tagesordnung" in x,
]
selected_ids = []
ids = []
for identifier in data_set_identifiers:
    for id, content in zip(antraege.document_id.values, antraege.content.values):
        for line in content:
             if identifier(line):
                ids.append(id)
                break
    selected_ids += np.random.choice(ids, size=1).tolist()


print(f"test set size {len(selected_ids)}")

dataset size 228
test set size 5


In [116]:
selector = np.isin(antraege.document_id, selected_ids)
for doc_id, content in iterate_content(doc_type="Antragsvorlage", document_ids=selected_ids):
    print(doc_id)
    result = remove_by_regexes(content, [
        r"Mit freundlichen Grüßen",
        r"gez.\s[A-Za-z]*",
    ], True)
    result = "".join(result)
    print("...", result[-100:])
    print("")
    print("\n".join(content)[-100:])

    print("=-----------------------------------------==")
    break

31766
... Wir sind davon überzeugt, dass Griesheim und seine Einwohner*innen dazu bereit und in der Lage sind.

rzeugt, dass Griesheim und seine Einwohner*innen dazu bereit und in der Lage sind.
gez. Martin Tichy
=-----------------------------------------==
